In [1]:
from os import path
import numpy as np
import matplotlib.pyplot as plt

from itertools import product
from tensorpac import Pac

from brainpipe.system import study
from brainpipe.feature import pac
from brainpipe.visual import *
from brainpipe.feat.utils._feat import _manageWindow

plt.style.use('seaborn-poster')

### Compute and Plot PAC values 
    Low frequency phases and large broad band gamma amplitude

In [2]:
st = study('Olfacto')
Test = False
if Test == True:
    subjects = ['FERJ']
    conds = {'subj':('LowHigh',['low','high'])}
else:
    subjects = ['FERJ','MICP','VACJ','SEMC','LEFC','PIRJ','CHAF']
    conds = {'subj':('LowHigh',['low','high'])}

-> Olfacto loaded


In [13]:
from brainpipe.feat.utils._feat import _manageWindow

for su, cond in product(subjects[:], sorted(conds)):
    pathdata = path.join(st.path,'database/Encoding_EpiPerf_LowHigh/')
    path2save = path.join(st.path,'feature/2_PAC_Encoding_EpiPerf_LowHigh/')
    # Define phase settings 
    fpha = [[3,7], [8,13]]
    famp = [[30,120]]
    fname = ['theta', 'alpha']
    
    for c in conds[cond][1]:
        # Load file :
        fi = su+'_odor_'+c+'_bipo_sel_physFT.npz'
        loadname = pathdata+fi
#         if path.exists(path2save+fi.replace('.npz','_pac.npz')):
#             print(fi.replace('.npz','_pac.npz'),' already computed')
#         else:
        mat = np.load(loadname)
        print(mat.files)
        data = mat['x']
        print(data.shape)
        n_elec, npts, n_trials = data.shape
        sf, width, step = 512.,358, 51
        print ('--> compute pac on : ', fi, 'data', data.shape, 'npts', npts, 'ntrials',n_trials, 'nelec', n_elec)              
        _, time = _manageWindow(data.shape[1], width=width, step=step)
        time = (np.array(time) / 512) - 3
        print(time, time[27:47])
        0/0
        #Compute pac, using the MVL method with surrogates and normalization (z-score)
        p = pac(sf, npts, Id='114', pha_f=fpha, pha_meth='hilbert', pha_cycle=3, 
                amp_f=famp,amp_meth='hilbert', amp_cycle=6, nbins=18, width=width,
                step=step,filtname='fir1')
        kwargs = {}
        kwargs['f'], kwargs['fname'], kwargs['famp'] = fpha, fname, famp
        kwargs['s_aal'], kwargs['s_BA'] = mat['aal'], mat['BA']
        kwargs['Mai_RL'], kwargs['channels'], kwargs['xyz'] = mat['Mai_RL'], mat['channels'], mat['xyz']
        xpac, _ = p.get(data,data,n_jobs=5) #(n_amplitude x n_phase x n_electrodes x n_windows x n_trials)
        kwargs['pac'] = np.squeeze(xpac)
        save_phase = path2save+fi.replace('.npz', '_pac.npz')
        np.savez(save_phase, **kwargs)

        del kwargs, data,xpac, sf, n_elec, n_trials

['x', 'Mai_RL', 'labels', 'BA', 'xyz', 'aal', 'sf', 'channels', 'Mai']
(85, 3584, 17)
--> compute pac on :  FERJ_odor_low_bipo_sel_physFT.npz data (85, 3584, 17) npts 3584 ntrials 17 nelec 85
[-2.65039062 -2.55078125 -2.45117188 -2.3515625  -2.25195312 -2.15234375
 -2.05273438 -1.953125   -1.85351562 -1.75390625 -1.65429688 -1.5546875
 -1.45507812 -1.35546875 -1.25585938 -1.15625    -1.05664062 -0.95703125
 -0.85742188 -0.7578125  -0.65820312 -0.55859375 -0.45898438 -0.359375
 -0.25976562 -0.16015625 -0.06054688  0.0390625   0.13867188  0.23828125
  0.33789062  0.4375      0.53710938  0.63671875  0.73632812  0.8359375
  0.93554688  1.03515625  1.13476562  1.234375    1.33398438  1.43359375
  1.53320312  1.6328125   1.73242188  1.83203125  1.93164062  2.03125
  2.13085938  2.23046875  2.33007812  2.4296875   2.52929688  2.62890625
  2.72851562  2.828125    2.92773438  3.02734375  3.12695312  3.2265625
  3.32617188  3.42578125  3.52539062  3.625     ] [ 0.0390625   0.13867188  0.23828125

ZeroDivisionError: division by zero

### Plot ST PAC values for all frequencies

In [5]:
pathdata = path.join(st.path,'feature/2_PAC_Encoding_EpiPerf_LowHigh/')
path2save = path.join(pathdata,'ST_PAC_plots/')

files = st.search('_pac.npz', folder=pathdata)
for fi in files:
    loadname = path.join(pathdata,fi)
    mat = np.load(loadname)
    xpac = mat['pac']
    n_elec, su = xpac.shape[1], fi[:4]
    for elec in range(n_elec):
        label = mat['Mai_RL'][elec]
        pac_elec = np.squeeze(xpac)[:,elec,:,:].swapaxes(0,2) #ntrials,n_pts,n_phase
        pac_elec = np.squeeze(np.mean(pac_elec,axis=1)) #mean pac value across time
        fig = plt.figure()
        plt.imshow(pac_elec, cmap='plasma',interpolation='none', origin ='lower', 
                   aspect='auto', vmin=0, vmax=2)
        plt.title(su+' PAC - '+fi[10:14]+' Memory '+label+ ' elec('+str(elec)+')')
        plt.xticks(np.arange(pac_elec.shape[1]),('theta','alpha'))
        plt.ylabel('Trials')
        plt.xlabel('Frequency for phase (Hz)')
        cb = plt.colorbar(ticks=[0,1,2],label='PAC')
        filename = fi.replace('sel_physFT_pac.npz',label+'_'+str(elec)+'_ST_pac.png')
        plt.savefig(path2save+filename)
        plt.tight_layout()
        plt.clf()
        plt.close()